# Collecting the Data

In [2]:
import pandas as pd
df2=pd.read_excel('FINAL_Waste_strat.xlsx')
df2

,Is this consultation being completed over the phone or uploaded as a paper copy? - Phone/paper,Is this consultation being completed over the phone or uploaded as a paper copy? - Analyst notes,Are you completing the consultation... - completing as,"If you are responding as an individual, which of the following best describes you:\n\n - indiv","If you are responding as an individual, which of the following best describes you:\n\n - Other (please state)","If you are responding on behalf of an organisation, please select which organisation type: - responding on behalf","If you are responding on behalf of an organisation, please select which organisation type: - Other:","If you are responding on behalf of an organisation, please select which organisation type: - Special interest group:",Does your organisation primarily work with or represent people from any of these groups? Please tick all that apply. - represent - Older people,Does your organisation primarily work with or represent people from any of these groups? Please tick all that apply. - represent - Children/young people,...,"Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Mental health needs","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Drug or alcohol addiction","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Physical impairment","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Learning difficulties/disabilities","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Autism spectrum disorder","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Other (please specify)","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - Disability - Prefer not to say","Do you consider yourself to have a health condition, disability or impairment? Please tick all that apply. - If other, please specify",Response ID,Submitted Date
0,Not Answered,NaN,as an individual,a private citizen,NaN,Not Answered,Not Answered,Not Answered,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANON-XKYM-2Z9Z-8,2023-09-13 09:29:21.945
1,Not Answered,NaN,as an individual,a private citizen,NaN,Not Answered,Not Answered,Not Answered,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANON-XKYM-2Z9Q-Y,2023-09-13 11:00:37.037
2,Not Answered,NaN,as an individual,a private citizen,NaN,Not Answered,Not Answered,Not Answered,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANON-XKYM-2Z96-4,2023-09-13 11:07:42.782
3,Not Answered,NaN,as an individual,a private citizen,NaN,Not Answered,Not Answered,Not Answered,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANON-XKYM-2Z9D-J,2023-09-13 12:06:52.812
4,Not Answered,NaN,as an individual,a private citizen,NaN,Not Answered,Not Answered,Not Answered,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANON-XKYM-2Z9K-S,2023-09-13 13:39:29.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4219,Paper copy,NaN,as an individual,Not Answered,NaN,Not Answered,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BHLF-XKYM-2M5W-M,2023-11-28 13:45:58.263
4220,Paper copy,NaN,as an individual,a private citizen,NaN,Not Answered,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BHLF-XKYM-2M57-M,2023-11-28 15:13:50.839
4221,Paper copy,NaN,as an individual,Not Answered,NaN,Not Answered,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BHLF-XKYM-2M5B-Y,2023-11-28 15:17:26.652
4222,Paper copy,NaN,as an individual,a private citizen,NaN,Not Answered,NaN,NaN,NaN,NaN,...,NaN,NaN,Physical impairment,NaN,NaN,NaN,NaN,NaN,BHLF-XKYM-2X9E-H,2023-11-28 15:40:28.558


# Data Pre-Processing

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt

## Lower Casing

In [4]:
df2 = df2.applymap(lambda s: s.lower() if type(s) == str else s)

## Initializing the Column Name

In [5]:
data = df2['Is there anything else you’d like to tell us about the priority or approach to delivering this priority?  - Comments']

In [6]:
data2=data

## Droping NaN Values

In [7]:
data2=data2.dropna() 

## Removing Special Characters

In [8]:
import re

# Remove special characters 
def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    # Replace special characters with empty string
    text = re.sub(pattern, '', text)
    return text

# Function to remove special characters 
data2 = data2.apply(remove_special_characters)

print(data2)

0       perhaps some recognition that in order to prot...
10      create working practices that encourage all to...
14      insufficient information on how this is going ...
15                great ambitions that are more realistic
24            this is just a pie in the sky shopping list
                              ...                        
4217    so what is the councils attitude to businesses...
4220    reading the preceding text and council proposa...
4221                     no to an incinerator in basildon
4222    a lot of fine words used but doesnt have a lot...
4223    drop charges for garden waste recycling  at a ...
Name: Is there anything else you’d like to tell us about the priority or approach to delivering this priority?  - Comments, Length: 1099, dtype: object


## Importing NLTK Library

In [9]:
# Import the nltk library
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# Tokenizing the data for certain range of rows
data2= data2.apply(lambda x: word_tokenize(x))


[nltk_data] Downloading package punkt to C:\Users\Rishwanth
[nltk_data]     Mithra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# Downloading the stopword package
nltk.download('stopwords')
# Importing the library
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Rishwanth
[nltk_data]     Mithra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Removing Stopwords using NLTK

In [11]:
# Function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
# Using the function to remove stopwords
data2= data2.apply(lambda x:remove_stopwords(x))

## Lemmatization

In [12]:
# Downloading the package
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
# Defining the object
wordnet_lemmatizer = WordNetLemmatizer()
# Defining the function
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
# Using the function for lemmatization
data2=data2.apply(lambda x:lemmatizer(x))


[nltk_data] Downloading package wordnet to C:\Users\Rishwanth
[nltk_data]     Mithra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
data2

0       [perhaps, recognition, order, protect, planet,...
10      [create, working, practice, encourage, support...
14      [insufficient, information, going, done, far, ...
15                           [great, ambition, realistic]
24                             [pie, sky, shopping, list]
                              ...                        
4217    [council, attitude, business, create, pollutio...
4220    [reading, preceding, text, council, proposal, ...
4221                              [incinerator, basildon]
4222    [lot, fine, word, used, doesnt, lot, meaning, ...
4223    [drop, charge, garden, waste, recycling, time,...
Name: Is there anything else you’d like to tell us about the priority or approach to delivering this priority?  - Comments, Length: 1099, dtype: object

## Vectorization

In [14]:
# create dictionary and corpus
# create dictionary
from gensim import corpora
id2word=corpora.Dictionary(data2)

#create corpus
corpus=[id2word.doc2bow(text) for text in data2]

# sample
print(corpus[2])

[(22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]


In [15]:
id2word

### Checking Word Frequency

In [16]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actionsthings', 1),
  ('cease', 1),
  ('granted', 1),
  ('order', 1),
  ('perhaps', 1),
  ('planet', 1),
  ('protect', 1),
  ('recognition', 1),
  ('take', 1)]]

# LDA Model

In [17]:
# Build LDA model
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=15,random_state=100,\
                                          update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [18]:
# print the 15 topics
lda_model.print_topics()


[(0,
  '0.052*"packaging" + 0.049*"used" + 0.049*"cant" + 0.045*"plan" + 0.043*"government" + 0.039*"look" + 0.039*"possible" + 0.037*"policy" + 0.036*"keep" + 0.028*"every"'),
 (1,
  '0.125*"recycle" + 0.111*"local" + 0.052*"centre" + 0.049*"system" + 0.047*"change" + 0.024*"provide" + 0.023*"information" + 0.022*"encouraging" + 0.022*"school" + 0.021*"everyone"'),
 (2,
  '0.092*"word" + 0.077*"resource" + 0.073*"achieve" + 0.046*"without" + 0.044*"big" + 0.038*"throw" + 0.036*"impact" + 0.032*"never" + 0.022*"poorer" + 0.019*"world"'),
 (3,
  '0.064*"energy" + 0.054*"also" + 0.038*"fly" + 0.036*"benefit" + 0.032*"tipping" + 0.030*"able" + 0.030*"currently" + 0.027*"large" + 0.027*"believe" + 0.026*"individual"'),
 (4,
  '0.065*"long" + 0.061*"pay" + 0.055*"example" + 0.052*"supporting" + 0.049*"enough" + 0.034*"job" + 0.029*"building" + 0.025*"working" + 0.023*"start" + 0.021*"industry"'),
 (5,
  '0.101*"ecc" + 0.088*"must" + 0.060*"collected" + 0.049*"cheaper" + 0.042*"education" + 

# Text Summarization

In [19]:
data3=data
data3

0       perhaps some recognition that in order to prot...
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
4219                                                  NaN
4220    reading the preceding text and council proposa...
4221                     no to an incinerator in basildon
4222    a lot of fine words used but doesn't have a lo...
4223    drop charges for garden waste recycling!  at a...
Name: Is there anything else you’d like to tell us about the priority or approach to delivering this priority?  - Comments, Length: 4224, dtype: object

## Droping NaN Values 

In [20]:
data3=data3.dropna()

In [21]:
data3

0       perhaps some recognition that in order to prot...
10      create working practices that encourage all to...
14      insufficient information on how this is going ...
15                great ambitions that are more realistic
24            this is just a pie in the sky shopping list
                              ...                        
4217    so what is the councils attitude to businesses...
4220    reading the preceding text and council proposa...
4221                     no to an incinerator in basildon
4222    a lot of fine words used but doesn't have a lo...
4223    drop charges for garden waste recycling!  at a...
Name: Is there anything else you’d like to tell us about the priority or approach to delivering this priority?  - Comments, Length: 1099, dtype: object

In [22]:
len(data3)

1099

In [36]:
#!pip install spacy


   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.2/12.1 MB 4.9 MB/s eta 0:00:03
   - -------------------------------------- 0.6/12.1 MB 5.8 MB/s eta 0:00:02
   -- ------------------------------------- 0.9/12.1 MB 6.2 MB/s eta 0:00:02
   -- ------------------------------------- 0.9/12.1 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/12.1 MB 7.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/12.1 MB 6.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/12.1 MB 6.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/12.1 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.1 MB 6.6 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.1 MB 6.7 MB/s eta 0:00:02
   ----------- ---------------------------- 3.5/12.1 MB 6.7 MB/s eta 0:00:02
   ------------ --------------------------- 3.8/12.1 MB 6.8 MB/s eta 0:00:02
   ---

## Importing Spacy

In [23]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [39]:
#!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 325.1 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 409.6 kB/s eta 0:00:32
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 2.2 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.8 MB 3.0 MB/s eta 0:00:04
     -- ------------------------------------- 0.8/12.8 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.8 MB 3.9 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.4 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.4 MB/s eta 0:00:03
     ------- -------------------------------- 2.4/12.8 MB 4.8 MB/s eta 0:00:03
     -------- ------------------------------- 2.6/12.8 MB 4.8 MB

In [24]:
nlp = spacy.load("en_core_web_sm")

In [25]:
d3=' '.join(data3)

In [26]:
doc=nlp(d3)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

## Lemmatization

In [28]:
#Lemmatization
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

## Removing Stopwords using spacy

In [29]:
from spacy.lang.en import English
parser = English()

In [30]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Rishwanth
[nltk_data]     Mithra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tokenizing the Data based on sentences

In [31]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [32]:
#Basic tokenizer
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

## Tokenized sentences

In [33]:
sent_token = [sent.text for sent in doc.sents]

In [35]:
len(sent_token)

1782

In [36]:
sent_token

['perhaps some recognition that in order to protect the planet some actions/things we take for granted should cease.',
 'create working practices that encourage all to support this and avoid punitive bureaucratic jobsworth creating policies and all will work better with you insufficient information on how this is going to be done!\n\n',
 'so far all i see is bull manure & not well rotted!',
 'great ambitions that are more realistic this is just a pie in the sky shopping list a considerable amount of advice and training for businesses and communities will be required to deliver these priorities, especially for small businesses.',
 'it makes perfect sense in theory.',
 'delivering will be much, much harder.',
 'for those of us that want to do the right thing, it will work and be easy.',
 "for those that don't care, they will carry on fly tipping or contaminating recyclables come what may unless there is strict enforcement.",
 'and if the enforcement means that my efforts get penalised th

## Grouping the sentences using scores of the Topic

In [34]:
# Testing Topic Modelling
new_doc = sent_token[13]
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = id2word.doc2bow(new_doc)
print(new_doc_bow)
print(lda_model.get_document_topics(new_doc_bow))

[(26, 1), (39, 1), (41, 2), (81, 1), (92, 1), (94, 1), (96, 1), (97, 1), (101, 1), (102, 1), (103, 1), (105, 1), (106, 1), (109, 1), (112, 1), (113, 1), (116, 1), (119, 2), (122, 1), (123, 1), (186, 1), (403, 1), (2038, 1), (2108, 1)]
[(0, 0.015196184), (1, 0.32669985), (3, 0.13611563), (4, 0.01066212), (7, 0.19196433), (9, 0.04649403), (10, 0.035592448), (11, 0.041680958), (12, 0.0988315), (13, 0.04977259), (14, 0.022592433)]


In [37]:
a=sent_token[333]+sent_token[338]+sent_token[339]+sent_token[379]#+sent_token[360]#+sent_token[371]+sent_token[399]#+sent_token[366]#+sent_token[102]+sent_token[103]+sent_token[299]#+sent_token[28]#+sent_token[29]+sent_token[30]+sent_token[31]

In [50]:
a 

'there is only so much the individual can do when we live in a waste producing society . there is a lot more the consumer can do to help support the planet and the targets outlined . if the bin is full he rips up into bits and puts the waste onto a box .'

### Summarizing the grouped sentences using Transformers

In [41]:
from transformers import pipeline

In [42]:
summarizer = pipeline("summarization", model='t5-base',tokenizer='t5-base',framework='pt')

In [48]:
summary = summarizer(a, max_length=200, min_length=40, do_sample=False)

Your max_length is set to 200, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


In [49]:
summary

[{'summary_text': 'there is only so much the individual can do when we live in a waste producing society . there is a lot more the consumer can do to help support the planet . if the bin is full he rips up into bits and puts the waste onto a box .'}]

Further, we have summarized the text into one action point through an API Call as discussed in the Summarization file.